<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-задания" data-toc-modified-id="Описание-задания-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание задания</a></span></li><li><span><a href="#Импорты" data-toc-modified-id="Импорты-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Импорты</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Repunctuation-and-recapitalization" data-toc-modified-id="Repunctuation-and-recapitalization-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Repunctuation and recapitalization</a></span><ul class="toc-item"><li><span><a href="#Пример" data-toc-modified-id="Пример-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Пример</a></span></li><li><span><a href="#Обработка-датафрейма" data-toc-modified-id="Обработка-датафрейма-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Обработка датафрейма</a></span></li></ul></li><li><span><a href="#NER" data-toc-modified-id="NER-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>NER</a></span><ul class="toc-item"><li><span><a href="#Natasha" data-toc-modified-id="Natasha-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span><del>Natasha</del></a></span></li><li><span><a href="#SpaCy:-ru_core_news_md-((natasha))" data-toc-modified-id="SpaCy:-ru_core_news_md-((natasha))-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>SpaCy: ru_core_news_md ((natasha))</a></span><ul class="toc-item"><li><span><a href="#SpaCy-+-RU2:" data-toc-modified-id="SpaCy-+-RU2:-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span><del>SpaCy + RU2:</del></a></span></li><li><span><a href="#ru_core_news_md:" data-toc-modified-id="ru_core_news_md:-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>ru_core_news_md:</a></span></li><li><span><a href="#Пример" data-toc-modified-id="Пример-5.2.3"><span class="toc-item-num">5.2.3&nbsp;&nbsp;</span>Пример</a></span></li><li><span><a href="#Обработка-датафрейма" data-toc-modified-id="Обработка-датафрейма-5.2.4"><span class="toc-item-num">5.2.4&nbsp;&nbsp;</span>Обработка датафрейма</a></span></li></ul></li></ul></li><li><span><a href="#ИТОГ" data-toc-modified-id="ИТОГ-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>ИТОГ</a></span></li><li><span><a href="##P.S.-Translate" data-toc-modified-id="#P.S.-Translate-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>#P.S. Translate</a></span><ul class="toc-item"><li><span><a href="#Пример" data-toc-modified-id="Пример-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Пример</a></span></li></ul></li></ul></div>

## Описание задания

1. Необходимо написать скрипт для парсинга диалогов из файла test_data.csv. Получившийся скрипт необходимо выложить в гит репозиторий и прислать ссылку в качестве результата прохождения тестового задания. Данные выкладывать в гит не следует. 
2. Главные задачи, которые должен выполнять скрипт:
    * Извлекать реплики с приветствием – где менеджер поздоровался. 
    * Извлекать реплики, где менеджер представил себя. 
    * Извлекать имя менеджера. 
    * Извлекать название компании. 
    * Извлекать реплики, где менеджер попрощался.
    * Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»
3. Рекомендации:
    * Сделать локальную копию файла test_data.csv, в исходнике никакие данные не менять!
    * Можно создать дополнительное поле в таблице test_data.csv, куда будет сохраняться результат парсинга – например, напротив реплики в столбце “insight” можно ставить флаг того, что эта реплика с приветствием greeting=True.
    * Для выполнения задачи можно использовать любые библиотеки и NLP модели. 
    * Попробуйте учесть возможные синонимичные выражения, которые могут помочь с извлечением данных сущностей.

## Импорты

In [1]:
from platform import python_version
python_version()

'3.7.8'

In [2]:
# !pip install torch

In [3]:
import os
import yaml
import torch
from torch import package
import pandas as pd
import re
import warnings

In [4]:
# !pip install spacy  ###==2.1.9

In [5]:
import spacy
spacy.__version__

'3.4.1'

In [6]:
# !python -m spacy download ru_core_news_md

In [7]:
from spacy.matcher import Matcher
# from spacy.pipeline import EntityRuler
# from spacy.pipeline import SpanRuler
from spacy import displacy
import ru_core_news_md

In [8]:
# !pip install -U deep-translator

In [9]:
from deep_translator import GoogleTranslator

In [10]:
warnings.filterwarnings("ignore", category=FutureWarning)

## Загрузка данных

In [11]:
df = pd.read_csv('test_data.csv')

In [12]:
df.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


## Repunctuation and recapitalization
# snakers4/silero-models
https://habr.com/ru/post/581960/

In [14]:
model, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                  model='silero_te')

Using cache found in C:\Users\skysh/.cache\torch\hub\snakers4_silero-models_master


### Пример

In [15]:
s_in = 'меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там'
print('in:', s_in)
print('out:', apply_te(s_in, lan='ru'))

in: меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там
out: Меня зовут Ангелина, компания, диджитал бизнес звоню вам по поводу продления лицензии, а мастера мы с вами сотрудничали по видео там.


### Обработка датафрейма

In [16]:
df['text'] = df['text'].str.lower().str.replace('+', 'плюс')
df.head(4)

,dlg_id,line_n,role,text
0,0,0,client,алло
1,0,1,manager,алло здравствуйте
2,0,2,client,добрый день
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...


In [17]:
%%time
df['text'] = df['text'].apply(lambda x: re.sub(r'(компания|фирма|товарищество|предприятие)(,)', r'\1',
                                               apply_te(x, lan='ru')))
df.head(4)

Wall time: 11.6 s


,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте!
2,0,2,client,Добрый день!
3,0,3,manager,"Меня зовут Ангелина, компания диджитал бизнес,..."


In [18]:
print(df.loc[3,'text'])

Меня зовут Ангелина, компания диджитал бизнес, звоним вам по поводу продления лицензии, а мы с серым у вас скоро срок заканчивается.


## NER

### ~~Natasha~~
https://habr.com/ru/post/516098/

Не удалось использовать из-за ошибки в Юпитере: "Kernel appeared to have died ..."

In [19]:
# !pip install pymorphy2
# !pip install razdel
# !pip install navec
# !pip install slovnet
# !pip install yargy
# !pip install ipymarkup
# !pip install pytest
# !pip install pytest-cov
# !pip install pytest-flakes
# !pip install pytest-pep8
# !pip install nbval

# !pip install natasha

In [20]:
# import pymorphy2
# import razdel
# import navec
# import slovnet
# import yargy
# import ipymarkup
# import pytest
# # import pytest-cov
# # import pytest-flakes
# # import pytest-pep8
# import nbval

In [21]:
# from natasha import (
#     Segmenter,
#     MorphVocab,
    
#     NewsEmbedding,
#     NewsMorphTagger,
#     NewsSyntaxParser,
#     NewsNERTagger,
    
#     PER,
#     NamesExtractor,

#     Doc
# )

In [22]:
# print(pymorphy2.__version__)  #0.8
# # print(razdel.__version__)  #0.5.0
# # print(navec.__version__)  #0.9.0
# # print(slovnet.__version__)  #0.3.0
# print(yargy.__version__)  #0.14.0
# # print(ipymarkup.__version__)  #0.8.0
# print(pytest.__version__)  #5.2.2
# # print(pytest-cov.__version__)  #2.8.1
# # print(pytest-flakes.__version__) #4.0.0
# # print(pytest-pep8.__version__) #1.0.6
# print(nbval.__version__) #0.9.5

In [23]:
# segmenter = Segmenter()
# morph_vocab = MorphVocab()

In [24]:
##### emb = NewsEmbedding()                        #### Kernel appeared to have died.
# # # # morph_tagger = NewsMorphTagger(emb)
# # # # syntax_parser = NewsSyntaxParser(emb)
# # # ner_tagger = NewsNERTagger(emb)
# # names_extractor = NamesExtractor(morph_vocab)

In [25]:
# text = 'Меня зовут Ангелина, компания диджитал бизнес, звоним вам по поводу продления лицензии, а мы с серым у вас скоро срок заканчивается.'
# doc = Doc(text)
# doc.segment(segmenter)
# # doc.tag_morph(morph_tagger)
# # doc.parse_syntax(syntax_parser)
# # doc.tokens
# doc.spans

### SpaCy: ru_core_news_md ((natasha))

#### ~~SpaCy + RU2:~~
https://github.com/buriy/spacy-ru

In [36]:
# nlp = spacy.load('ru2')

#### ru_core_news_md:

In [26]:
nlp = ru_core_news_md.load()
## ru_core_news_lg, ru_core_news_md, ru_core_news_sm
# nlp = spacy.load("en_core_web_sm")

nlp.add_pipe('sentencizer', first=True)
# nlp.add_pipe(nlp.create_pipe('sentencizer'), first=True)

patterns = [{"label": "ORG", "pattern": [{"LOWER": {"IN": ["компания", "фирма", 
                                                            "товарищество", "предприятие"]}},
                                         {"POS": {"IN": ["VERB", "ADJ"]}, "OP": "*"},
                                         {"POS": {"IN": ["PROPN", "NOUN"]}, "OP": "+"}
                                        ]
            },
           {"label": "Greetings", "pattern": [{"LEMMA": "добрый"},  #, "IS_SENT_START": True},
                        {"LEMMA": {"IN": ["утро","день","вечер","ночь","время"]}, "OP":"?"}
                        ]
           },
            {"label": "Greetings", "pattern": [{"NORM": {"IN":["здравствуйте", "привет"]}}]},
            {"label": "BYE", "pattern": [{"NORM":"до"}, {"NORM": {"IN":["свидания","встречи"]}}]},
            {"label": "BYE", "pattern": [{"NORM":"всего"}, {"NORM": {"IN":["доброго","хорошего"]}}]},
            
           ]
ruler = nlp.add_pipe("entity_ruler", after='ner', config={'overwrite_ents': True})
ruler.add_patterns(patterns)

### elder version of adding pipe in SpaCy 2.1.9 (for RU2):
# ruler = EntityRuler(nlp, overwrite_ents=True)
# nlp.add_pipe(ruler, after='ner') ##, config={'overwrite_ents': True})

## span_ruler = nlp.add_pipe("span_ruler")
## print(nlp.pipe_names)

#### Пример

In [27]:
df.at[111,'text']

'Меня зовут Ангелина, компания диджитал бизнес звоню вам по поводу продления, а мы сели обратила внимание, что у вас срок заканчивается.'

In [28]:
# sample_sentences = df.at[111,'text']
sample_sentences = 'компания диджитал бизнес звоню'
doc = nlp(sample_sentences)

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print("Сущности: ", ents)

for t in doc:
    if t.ent_type_ == 'PER':
        print(t.text, t.dep_, t.prob, t.ent_id)
        ancestor_list = list(t.ancestors)
        if len(ancestor_list) > 0:
            ancestor = ancestor_list[-1]
            print("  ", ancestor.lemma_, ancestor.pos_)
            if ancestor.lemma_ == 'звать':
                df.loc[i,'manager_name'] = t.lemma_
#                 name_flag = True
        lefts_list = list(t.lefts)
        if len(lefts_list) > 0:
            left = lefts_list[-1]
            if left.lemma_ == 'это' and left.dep_ == 'nsubj':
                print("   ", left.lemma_, left.pos_, left.dep_)

Сущности:  [('компания диджитал бизнес звоню', 0, 30, 'ORG')]


In [29]:
### Визуализация синтаксического разбора:
# displacy.serve(doc, style="dep")

#### Обработка датафрейма

In [30]:
# %%time
# df['entities'] = df.text.apply(lambda x: [(e.text, e.start_char, e.end_char, e.label_) for e in nlp(x).ents])
# for ind in range(len(df)):
#     if df.loc[ind,'entities'] != []:
#         print(df.loc[ind,'text'])
#         print(df.loc[ind,'entities'], '\n')

In [31]:
df['entities'] = None
df['manager_name'] = None
df['company_name'] = None
df['greetings'] = None
df['goodbye'] = None

In [32]:
dlg_id = -1
greetings_flag = False
goodbye_flag = False
org_flag = False
name_flag = False
for i in range(df.shape[0]):  #range(6)
    if df.loc[i,'dlg_id'] > dlg_id:
        greetings_flag = False
        goodbye_flag = False
        org_flag = False
        name_flag = False
        dlg_id = df.loc[i,'dlg_id']
    if df.loc[i,'role'] == 'client':
        continue
        
    doc = nlp(df.loc[i,'text'])
    ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
    df.at[i,'entities'] = ents
    if name_flag == False:
        for t in doc:
            if t.ent_type_ == 'PER':
#                 print(t.text, t.dep_, t.prob, t.ent_id)
                ancestor_list = list(t.ancestors)
                if len(ancestor_list) > 0:
                    ancestor = ancestor_list[-1]
    #                 print("  ", ancestor.lemma_, ancestor.pos_)
                    # Варианты фраз типа "Меня зовут UserName.":
                    if ancestor.lemma_ == 'звать':
                        df.at[i,'manager_name'] = t.lemma_.capitalize()
                        name_flag = True
                lefts_list = list(t.lefts)
                if len(lefts_list) > 0:
                    left = lefts_list[-1]
                    # Варианты фраз типа: "Это UserName."
                    if left.lemma_ == 'это' and left.dep_ == 'nsubj':
#                         print("   ", left.lemma_, left.pos_, left.dep_)
                        df.at[i,'manager_name'] = t.lemma_.capitalize()
                        name_flag = True
    for e in doc.ents:
        if e.label_ == 'ORG' and org_flag == False and df.at[i,'manager_name'] is not None:
            # Компанию называют в том же высказывании, что и имя. Искать дальше по тексту чревато ошибками.
            df.at[i,'company_name'] = re.sub(r'(компания|фирма|товарищество|предприятие)', '', e.lemma_).strip()
            org_flag = True
        if e.label_ == 'Greetings' and greetings_flag == False:
            df.at[i,'greetings'] = e.text
            greetings_flag = True
        if e.label_ == 'BYE' and goodbye_flag == False:
            df.at[i,'goodbye'] = e.text
            goodbye_flag = True

## ИТОГ

In [33]:
df.drop(columns=['role','text','entities']).query("manager_name.notnull() or greetings.notnull() or company_name.notnull() or goodbye.notnull()")

,dlg_id,line_n,manager_name,company_name,greetings,goodbye
1,0,1,None,None,здравствуйте,None
3,0,3,Ангелина,диджитал бизнес,None,None
108,0,108,None,None,None,Всего хорошего
110,1,1,None,None,здравствуйте,None
111,1,2,Ангелина,диджитал бизнес звоню,None,None
162,1,53,None,None,None,всего хорошего
166,2,2,None,None,здравствуйте,None
167,2,3,Ангелина,диджитал бизнес звоню,None,None
250,3,1,None,None,добрый день,None
251,3,2,Максим,китобизнес,None,None


In [41]:
df_resume = df.groupby('dlg_id')['manager_name', 'company_name', 'greetings', 'goodbye'].count()#.reset_index(drop=False)
df_resume['greetings_and_bye'] = df_resume['greetings'] * df_resume['goodbye']
df_resume

,manager_name,company_name,greetings,goodbye,greetings_and_bye
dlg_id,,,,,
0,1,1,1,1,1
1,1,1,1,1,1
2,1,1,1,0,0
3,1,1,1,1,1
4,0,0,0,1,0
5,1,0,0,1,0


Есть ошибки в вычленении названия компании, связанные с ошибками синтаксического разбора. Можно их попробовать нейтрализовать при помощи span_ruler, например. Не успела разобраться пока.

## #P.S. Translate

*Можно поэкспериментировать ещё как-то, например: предобработка в виде перевода + NER на английском.*

### Пример

In [35]:
translator = GoogleTranslator(source='auto', target='en')
translated = translator.translate("Меня зовут Ангелина компания диджитал бизнес звоним вам по поводу продления лицензии, а мы с серым у вас скоро срок заканчивается.")
translated

'My name is Angelina, a digital business company, we are calling you about renewing a license, and gray and I, your term is ending soon.'